# HMI資料合併與特徵工程

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
import os
import math
import time
import datetime
from scipy.stats import kurtosis,skew#new
from scipy.fftpack import fft#new

In [2]:
# 這裡是存這個資料夾底下的資料夾並存成一個list因為我們將每天的資料都存成一個資料夾
path='D:\\桌面\\before_Combine\\'
dir_list = os.listdir(path)
dir_list

['0805', '0806', '0807', '0808', '0809', '0810', '0811', '0812']

In [3]:
# 把 HMI狀態的json檔讀近來然後拆分塞進dataframe
read_HMI_json = pd.read_json('HMI_data.json')
get_Data_origin = read_HMI_json['data']
get_Data_twice = get_Data_origin['history']
df_HMI_json = DataFrame(get_Data_twice,columns = ['attrId','name','unit','values'])

In [4]:
# 將拆好的dataframe再拆成三個dataframe，分別為機台狀態、溫度、電流。
df_HMI_temperature = DataFrame(df_HMI_json.iloc[0,3])
df_HMI_status = DataFrame(df_HMI_json.iloc[1,3])
df_HMI_current = DataFrame(df_HMI_json.iloc[2,3])
df_HMI_shape = df_HMI_status.shape

#把時間戳換成datetime格式，並寫回去到原本的DF
for timestamp in range(df_HMI_shape[0]):
    df_HMI_status.iloc[timestamp,1] = datetime.datetime.fromtimestamp(round(df_HMI_status.iloc[timestamp,1]/1000))

In [5]:
# 檢查一下dataframe
df_HMI_status

,value,reportTime,updateTime
0,0,2020-08-04 03:06:01,1596487537892
1,0,2020-08-04 03:06:31,1596487539900
2,0,2020-08-04 03:07:01,1596487539932
3,0,2020-08-04 03:07:31,1596487539977
4,0,2020-08-04 03:08:02,1596487540895
...,...,...,...
32264,0,2020-08-15 07:57:32,1597449453163
32265,0,2020-08-15 07:58:01,1597449482996
32266,0,2020-08-15 07:58:31,1597449512818
32267,0,2020-08-15 07:59:02,1597449543162


In [6]:
# 將HMI狀態的時間都抓出來做成一個list但只做到分鐘，秒都用00去替代
datetime_list = []
dirdaytime_list = []
# 將HMI的每個資料的時間抓出來做日期格式化並存成list備用
for datetime_format in range(df_HMI_shape[0]) :
    dirdaytime_list.append(df_HMI_status.iloc[datetime_format,1].strftime('%m%d'))
    datetime_list.append(df_HMI_status.iloc[datetime_format,1].strftime('%Y-%m-%d %H:%M')+':00')
# 抓好之後將list中不重複的抓出來做成一個list備用
datetime_list_unique = np.unique(datetime_list)
dirdaytime_list_unique = np.unique(dirdaytime_list)
# 將上面的HMI狀態時間list的shape存起來備用
datetime_list_shape = len(datetime_list_unique)

In [7]:
# 定義一個function來抓時間列表
# 輸入是一分鐘的string 例如:2020-08-05 00:01:00
# txt_Time_list是上面HMI資料的所有格式化到小時的不重複資料
def find_Index_wrong_position_list(correct_Time,txt_Time_list):
    # 看HMI機台不重複時間有多少資料
    def_len =  len(txt_Time_list)
    # 開一個新的list準備儲存需要的時間
    correct_list = []
    # 將輸入進來的時間格式化
    time_Format_c = datetime.datetime.strptime(correct_Time,"%Y%m%d%H")
    time_Format_c = time_Format_c.strftime("%Y%m%d%H")
    print(time_Format_c)
    # 這邊是為了讓程式更謹慎設定的count跟flag
    flag = 0
    check_count = 0
    # for迴圈跑每一筆txt_Time_list時間
    for def_line in range(def_len-1):
        # 將txt_Time_list的時間資料做格式化
        txt_time_check = datetime.datetime.strptime(str(txt_Time_list[def_line]),"%Y-%m-%d %H:%M:%S")
        txt_time_check = txt_time_check.strftime("%Y%m%d%H")
        # 比對格式化完的資料是不是一樣，如果一樣就存入list 同時比對下一個時間是否也一樣
        # 如果不一樣就直接回傳list
        if txt_time_check == time_Format_c:
            correct_list.append(txt_Time_list[def_line])
            next_time = datetime.datetime.strptime(str(txt_Time_list[def_line+1]),"%Y-%m-%d %H:%M:%S")
            next_time = next_time.strftime("%Y%m%d%H")
            if next_time != time_Format_c:
                return correct_list
                flag += 2
        # 這一段用不到
        elif txt_time_check != time_Format_c and flag==2:
            return correct_list
        # 這邊做檢查點
        elif txt_time_check != time_Format_c and flag==0:
            check_count+=1
        else:
            print('Error function find_Index_wrong_position_list() has a bug')

# 切分(新)

In [8]:
def cut_Sec_df_Ver3(df_Cut_Sec_before):
    # 將dataframe的長度存入一個變數備用
    df_Shape_min = df_Cut_Sec_before.shape
    # 把需要的資料欄位都開一個list
    sec_Timestamp = []
    sec_Z1_mean = []
    sec_Z2_mean = []
#   --------------------------------------------------------------------------------------
    sec_Z1_max = []
    sec_Z1_min = []
    sec_Z2_max = []
    sec_Z2_min = []
#   --------------------------------------------------------------------------------------
    sec_Z1_std = [] # 標準差
    sec_Z1_skewness = [] # 偏度
    sec_Z1_kurtosis = [] # 峰度
    
    sec_Z1_peakTopeak = [] # 波峰到波谷質
    sec_Z1_RMS = [] # 平方平均數
    sec_Z1_crestFactor = [] #峰質因素
    sec_Z1_shapeFactor = [] #塑形工程
    sec_Z1_impulseFactor = [] #脈衝工程
    sec_Z1_marginFactor = [] #邊界工程

    Spectrum_mean=[]#頻譜平均
    Spectrum_std = []#頻譜標準差
    Spectrum_mad = []#頻譜偏差
    Spectrum_kur = []#頻譜峰度
    Energy_signal = []#訊號能量
#   --------------------------------------------------------------------------------------
#   --------------------------------------------------------------------------------------
    tmp_datatime = pd.to_datetime(df_Cut_Sec_before.iloc[0,0])
#     print(type(tmp_datatime))
#     print((tmp_datatime))
#     now = (tmp_datatime+datetime.timedelta(seconds=1)).strftime("%Y-%m-%d %H:%M:%S")
    sec_Timestamp.append(tmp_datatime)
    df_sec_25 = pd.DataFrame()
    df_sec_50 = pd.DataFrame()
    df_sec_75 = pd.DataFrame()
        
    #df_sec的四分位數
    df_sec25 = df_Cut_Sec_before.quantile(0.25)
    df_sec75 = df_Cut_Sec_before.quantile(0.75)

    df_sec_quartile25 = df_sec25['Z1']
    df_sec_quartile75 = df_sec75['Z1']
    #排序資料後用前25%的資料取最小值 最後25%取最大值 剩下的取平均值
    df_sec_25 = df_sec_25.append(df_Cut_Sec_before[df_Cut_Sec_before['Z1']<=df_sec_quartile25],ignore_index=True)
    df_sec_50 = df_sec_50.append(df_Cut_Sec_before[(df_Cut_Sec_before['Z1']>df_sec_quartile25) & (df_Cut_Sec_before['Z1']<df_sec_quartile75)],ignore_index=True)
    df_sec_75 = df_sec_75.append(df_Cut_Sec_before[df_Cut_Sec_before['Z1']>=df_sec_quartile75],ignore_index=True)
    #將做好最大最小平均的資料放進list
    sec_Z1_max.append(np.max(df_sec_75['Z1']))
    sec_Z1_min.append(np.min(df_sec_25['Z1']))
    sec_Z1_mean.append(np.mean(df_sec_50['Z1']))
    #將Z1的標準差 偏度 峰度 波峰到波谷值 放進list(這裡是直接呼叫函數直接運算後直接存入list)
    sec_Z1_std.append(np.std(df_Cut_Sec_before['Z1']))
    sec_Z1_skewness.append(skew(df_Cut_Sec_before['Z1']))
    sec_Z1_kurtosis.append(kurtosis(df_Cut_Sec_before['Z1']))
    sec_Z1_peakTopeak.append(np.ptp(df_Cut_Sec_before['Z1']))
    #計算其他特徵工程需要的數字
    RMS = math.sqrt(pow(np.mean(df_Cut_Sec_before['Z1']),2)+pow(np.std(df_Cut_Sec_before['Z1']),2))
    crestFactor = np.max(np.abs(df_Cut_Sec_before['Z1'])/np.sqrt(np.mean(np.square(df_Cut_Sec_before['Z1']))))
    shapeFactor = RMS/np.mean(np.abs(df_Cut_Sec_before['Z1']))
    impulse = np.max(df_Cut_Sec_before['Z1'])/np.abs(np.mean(df_Cut_Sec_before['Z1']))
    margin = np.max(df_Cut_Sec_before['Z1'])/pow(np.mean(np.abs(df_Cut_Sec_before['Z1'])),2)
    #算完就放入list中
    sec_Z1_RMS.append(RMS)
    sec_Z1_crestFactor.append(crestFactor)
    sec_Z1_shapeFactor.append(shapeFactor)
    sec_Z1_impulseFactor.append(impulse)
    sec_Z1_marginFactor.append(margin)
    
    df_Z1 = df_Cut_Sec_before['Z1']
    df_Z1 = df_Z1.reset_index(drop=True)
    #這一塊在做頻譜計算
    Hz = df_Shape_min[0]/60
    # 要修改的FFT
    def FFT(x,Hz):
        return np.sin(2*np.pi*Hz*x)
    dataY = np.zeros(df_Shape_min[0])

    for j in range(0,df_Shape_min[0]):
        dataY[j] = FFT(df_Z1[j],Hz)
    #算完就直接放入list
    Spectrum = np.abs(fft(dataY))
    Spectrum_mean.append(np.mean(Spectrum))
    Spectrum_std.append(np.std(Spectrum))
    Spectrum_mad.append(skew(Spectrum))
    Spectrum_kur.append(kurtosis(Spectrum))
    Energy_signal.append((np.mean(df_Z1)**2))
#   --------------------------------------------------------------------------------------
#     min_Z1_max = np.max(sec_Z1_max)
#     min_Z1_min = np.min(sec_Z1_min)
#     min_Z2_max = np.max(sec_Z2_max)
#     min_Z2_min = np.min(sec_Z2_min)
#   --------------------------------------------------------------------------------------
    # 開一個新的dataframe 將上面做的特徵工程list存入dataframe
    sec_point = pd.DataFrame()
    sec_point['Timestamp'] = sec_Timestamp
    sec_point['Z1_mean'] = sec_Z1_mean
    sec_point['Z1_max'] = sec_Z1_max
    sec_point['Z1_min'] = sec_Z1_min
    sec_point['Z1_std'] = sec_Z1_std
    sec_point['Z1_skew'] = sec_Z1_skewness
    sec_point['Z1_kurtosis'] = sec_Z1_kurtosis
    sec_point['Z1_peakTopeak'] = sec_Z1_peakTopeak
    sec_point['Z1_RMS'] = sec_Z1_RMS
    sec_point['Z1_crestFactor'] = sec_Z1_crestFactor
    sec_point['Z1_shapeFactor'] = sec_Z1_shapeFactor
    sec_point['Z1_impulseFactor'] = sec_Z1_impulseFactor
    sec_point['Z1_marginFactor'] = sec_Z1_marginFactor
    sec_point['energy_Signal'] = Energy_signal
    sec_point['spectrum_Mean'] = Spectrum_mean
    sec_point['spectrum_Std'] = Spectrum_std
    sec_point['spectrum_Skew'] = Spectrum_mad
    sec_point['spectrum_Kur'] = Spectrum_kur
#     sec_point['spectrum'] = Spectrum
    # 將dataframe回傳
    return sec_point

In [9]:
# for迴圈跑每個資料
for dirdaytime_list_unique in dir_list:
    # 先創一個新的dataframe，這是用來儲存要輸出的資料
    csv_Dataframe = pd.DataFrame()
    
#     path='D:\\福壽\\製粒機\\製粒機(8月)\\'
    path='D:\\桌面\\before_Combine\\'
    # 用上面的資料夾加上路徑直接抓到那一天資料夾底下的資料
    file_list = os.listdir(path+dirdaytime_list_unique)
    # 抓小時
    # 資料夾內的每個檔案都跑一遍
    for read_File_list in file_list :
        # 將讀到的資料存成dataframe
        df_Clean_json = pd.read_json(path+dirdaytime_list_unique+'\\'+read_File_list)
#         txt_Time_index=999999999
        count_minute = 0
        # 將檔名多餘的字元切除，把檔名中的時間當作索引然後帶入寫好的function後會得到這個小時在HMI資料中所有的時間列表
        file_list_time = read_File_list.split('_')
        # datetime_list_unique是上面在處理HMI資料時做的
        time_After_check = find_Index_wrong_position_list(file_list_time[0],datetime_list_unique)
        # 如果time_After_check中沒有任何資料代表這個小時的檔案中沒有任何一個時間是在HMI資料的範圍內
        if time_After_check == None:
            print('這小時沒有任何資料喔~啾咪 0.<')
        else:
            # 將time_After_check的每個時間都做for迴圈處理(每分鐘都要做處理)
            for every_Minute_in_txt in time_After_check:
                # 將dataframe中時間等於for迴圈讀到的那一分鐘的所有資料存入另一個dataframe
                df_sec_1 = df_Clean_json.loc[df_Clean_json['Timestamp']== str(every_Minute_in_txt)]
                try:
                    # 確認這一分鐘在HMI中的資料有幾筆
                    index_count = datetime_list.count(str(df_sec_1.iloc[0,0]))
                    # 然後將新的dataframe放進特徵工程用的function
                    df_60_sec = cut_Sec_df_Ver3(df_sec_1)
                except Exception as e:
                    print('Error - 特徵工程function ',e)
                count_minute+=1
                # 設定if條件，正常狀態的HMI一分鐘會有兩個資料，但有些會有3個，所以這邊做檢查。
                if index_count == 3:
                    try:
                        # 這邊是用df_sec_1的這一分鐘當作索引去找HMI機台狀態中這一分鐘的索引值
                        txt_Time_index = datetime_list.index(str(df_sec_1.iloc[0,0]))
                        print(txt_Time_index)
                        #----------------------------------------------------------------------------
                        # 在dataframe中開新的欄位準備合併HMI機台資料
                        df_60_sec['machine_Status_avg']=''
                        df_60_sec['Temperature_avg']=''
                        df_60_sec['Current_avg']=''
                        df_60_sec['Current_max']=''
                        # 將HMI機台狀態的資料都存成list
                        machine_Status_list = [float(df_HMI_status.iloc[txt_Time_index,0]),float(df_HMI_status.iloc[txt_Time_index+1,0]),float(df_HMI_status.iloc[txt_Time_index+2,0])]
                        Temperature_list = [float(df_HMI_temperature.iloc[txt_Time_index,0]),float(df_HMI_temperature.iloc[txt_Time_index+1,0]),float(df_HMI_temperature.iloc[txt_Time_index+2,0])]
                        Current = [float(df_HMI_current.iloc[txt_Time_index,0]),float(df_HMI_current.iloc[txt_Time_index+1,0]),float(df_HMI_current.iloc[txt_Time_index+2,0])]
                        #----------------------------------------------------------------------------
                        # 將存成list的HMI機台狀態做 max跟平均 然後存在dataframe後面
                        df_60_sec.iloc[:,18] = np.max(machine_Status_list)
                        #----------------------------------------------------------------------------
                        df_60_sec.iloc[:,19] = np.mean(Temperature_list)
                        #----------------------------------------------------------------------------
                        df_60_sec.iloc[:,20] = np.mean(Current)
                        df_60_sec.iloc[:,21] = np.max(Current)
                        #----------------------------------------------------------------------------
                        # 將寫好的dataframe併入上面開的空dataframe，等到換資料夾時會直接輸出成csv再清空。
                        csv_Dataframe = csv_Dataframe.append(df_60_sec,ignore_index=True)
                    except Exception as q:
                        print('index_count = 3 , index_count判定有bug: ',q)
#                     txt_Time_index = datetime_list.index(df_Clean_json['Timestamp'].iloc[0])
                    #----------------------------------------------------------------------------
                    print('index_count = 3')
                # 下面的if條件的code都跟上面一樣，只差在HMI有幾個值要放進list
                elif index_count == 2:
                    try:
                        txt_Time_index = datetime_list.index(str(df_sec_1.iloc[0,0]))
#                         print(str(df_sec_1.iloc[0,0]),txt_Time_index)
                        #----------------------------------------------------------------------------
                        df_60_sec['machine_Status_avg']=''
                        df_60_sec['Temperature_avg']=''
                        df_60_sec['Current_avg']=''
                        df_60_sec['Current_max']=''
                        machine_Status_list = [float(df_HMI_status.iloc[txt_Time_index,0]),float(df_HMI_status.iloc[txt_Time_index+1,0])]
                        Temperature_list = [float(df_HMI_temperature.iloc[txt_Time_index,0]),float(df_HMI_temperature.iloc[txt_Time_index+1,0])]
                        Current = [float(df_HMI_current.iloc[txt_Time_index,0]),float(df_HMI_current.iloc[txt_Time_index+1,0])]
                        #----------------------------------------------------------------------------
                        df_60_sec.iloc[:,18] = np.max(machine_Status_list)
                        #----------------------------------------------------------------------------
                        df_60_sec.iloc[:,19] = np.mean(Temperature_list)
                        #----------------------------------------------------------------------------
                        df_60_sec.iloc[:,20] = np.mean(Current)
                        df_60_sec.iloc[:,21] = np.max(Current)
                        #----------------------------------------------------------------------------
                        csv_Dataframe = csv_Dataframe.append(df_60_sec,ignore_index=True)
                    except Exception as q:
                        print('index_count = 2 , 判定有bug: ',q)
                elif index_count == 1:
                    try:
                        txt_Time_index = datetime_list.index(str(df_sec_1.iloc[0,0]))
                        #----------------------------------------------------------------------------
                        df_60_sec['machine_Status_avg']=''
                        df_60_sec['Temperature_avg']=''
                        df_60_sec['Current_avg']=''
                        df_60_sec['Current_max']=''
                        machine_Status_list = [float(df_HMI_status.iloc[txt_Time_index,0])]
                        Temperature_list = [float(df_HMI_temperature.iloc[txt_Time_index,0])]
                        Current = [float(df_HMI_current.iloc[txt_Time_index,0])]
                        #----------------------------------------------------------------------------
                        df_60_sec.iloc[:,18] = machine_Status_list
                        #----------------------------------------------------------------------------
                        df_60_sec.iloc[:,19] = Temperature_list
                        #----------------------------------------------------------------------------
                        df_60_sec.iloc[:,20] = Current
                        df_60_sec.iloc[:,21] = Current
                        #----------------------------------------------------------------------------
                        csv_Dataframe = csv_Dataframe.append(df_60_sec,ignore_index=True)
                    except Exception as q:
                        print('index_count = 1 , 判定有bug: ',q)
                        #----------------------------------------------------------------------------
                else:
                    print('Error 其他:',df_sec_1.iloc[0,0],'index_count:',index_count)
            # 提示進度
            print(read_File_list,count_minute,index_count)
    # 寫入csv
    csv_Dataframe.to_csv('D:\\桌面\\train_Data\\json_Combine_txt_ToCSV_'+dirdaytime_list_unique+'_V3.csv',index=False)

2020080500
2020080500_after.json 60 2
2020080501
2020080501_after.json 60 2
2020080502
2020080502_after.json 60 2
2020080503
2020080503_after.json 60 2
2020080504
2020080504_after.json 60 2
2020080505
2020080505_after.json 60 2
2020080506
2020080506_after.json 60 2
2020080507
2020080507_after.json 60 2
2020080508
2020080508_after.json 60 2
2020080509
2020080509_after.json 60 2
2020080510
2020080510_after.json 60 2
2020080511
2020080511_after.json 60 2
2020080512
4015
index_count = 3
2020080512_after.json 60 2
2020080513
4184
index_count = 3
2020080513_after.json 60 2
2020080514
2020080514_after.json 60 2
2020080515
2020080515_after.json 60 2
2020080516
2020080516_after.json 60 2
2020080517
2020080517_after.json 60 2
2020080518
2020080518_after.json 60 2
2020080519
2020080519_after.json 60 2
2020080520
2020080520_after.json 60 2
2020080521
2020080521_after.json 60 2
2020080522
2020080522_after.json 60 2
2020080523
2020080523_after.json 60 2
2020080600
2020080600_after.json 60 2
20200806

KeyboardInterrupt: 

In [ ]:
print(time_After_check)

In [ ]:
for i in datetime_list_unique:
    print(i)